In [1]:
import numpy as np
import cv2

In [2]:
import sys
import os

# Get the absolute path to the parent directory containing "02504 Computer Vision"
parent_dir = os.path.abspath(os.path.join("../../..", "02504-Computer-Vision"))

# Add it to sys.path
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from utility import Pi, Piinv

![qn1](images/qn1.png)
![mat](images/intrin.png)

In [3]:
f = 1200
principal_point = (400, 350)
cx = principal_point[0]
cy = principal_point[1]
alpha = 1
beta = 0
k3 = 0.01
k5 = 0.04
K = np.asarray([[f, beta * f, cx], [0, alpha * f, cy], [0, 0, 1]])
K

array([[1200,    0,  400],
       [   0, 1200,  350],
       [   0,    0,    1]])

![qn2](images/qn2..png)

![ans2](images/ans2.png)

## Extra functions from ex2

![distortion](images/dist_formula.png)

In [4]:
def dist(point: np.array, distCoeffs: np.array, poly_func):
    """
    Returns a distorted version of x based on radial distortion
    With polynomial coefficients in distCoeffs

    Args:
        point (np.array): Input inhomogeneous 2d points
        distCoeffs (np.array): radial distortion polynomial coefficients
        poly_func (function): Function that computes polynomial

    Return:
        np.array of distorted image in inhomogeneous coordinates
    """
    point_x = point[0]
    point_y = point[1]
    squared = np.sqrt(point_x**2 + point_y**2)
    print(f"Undergoing distortion with coeffs: {distCoeffs}")
    distortion_factor = poly_func(squared, distCoeffs)
    distorted_point = point * distortion_factor
    return distorted_point


distCoeffs = np.asarray([0.1, 0.01])
point = np.asarray([1.0, 2.0])
poly_lambda = lambda squared, coeffs: 1 + coeffs[0] * squared + coeffs[1] * (squared**2)
distorted_point = dist(point, distCoeffs, poly_lambda)
distorted_point

Undergoing distortion with coeffs: [0.1  0.01]


array([1.2736068, 2.5472136])

## Undistortion

In [5]:
def undistortImage(img: np.array, K: np.array, distCoeffs: np.array):
    """
    Undistorts image, main principle is as follows:
    1. Create an empty grid to store all the mappings from camera coordinate system -> distorted pixel coordinates
    2. Get image in terms of camera coordinate by multiplying grid with the inverse of camera intrinsics
    3. Distort the grid with the given radial distortion polynomial and distortion coefficients
       Result would give the mapping from camera coordinate -> Distorted pixel coordinate
    4. Use remap() with interpolation for non integer pixel coordinates

    Basically create an empty canvas, let it go through all the transformations (e.g. camera -> pixel + distortion),
    then find mapping between this transformed canvas and the original distorted image

    Args:
        img (np.array): distorted image
        K (np.array): intrinsics matrix
        distCoeffs (np.array): distortion coefficient matrix

    Return:
        im_undistorted (np.array): undistorted image
    """
    x, y = np.meshgrid(np.arange(img.shape[1]), np.arange(img.shape[0]))
    # Stack both and add ones to make p homogeneous
    p = np.stack((x, y, np.ones(x.shape))).reshape(3, -1)
    q = np.linalg.inv(K) @ p
    poly_lambda = (
        lambda squared, coeffs: 1
        + coeffs[0] * (squared**2)
        + coeffs[1] * (squared**4)
        + coeffs[2] * (squared**6)
    )
    q_d = dist(Pi(q), distCoeffs, poly_lambda)
    p_d = K @ Piinv(q_d)
    # New distorted image
    x_d = p_d[0].reshape(x.shape).astype(np.float32)
    y_d = p_d[1].reshape(y.shape).astype(np.float32)
    assert (p_d[2] == 1).all(), "You did a mistake somewhere"
    # Find remapping
    im_undistorted = cv2.remap(img, x_d, y_d, cv2.INTER_LINEAR)
    return im_undistorted